In [1]:
import numpy as np
import cv2

import os
import glob
from scipy.ndimage import rotate

In [2]:
DIM = (1280, 1280)

In [13]:
def mapping(x, y):
    theta = 2.0 * np.pi * (x / 2560 - 0.5) # -pi to pi
    phi = np.pi * (y / 1280 - 0.5)	# -pi/2 to pi/2
    r = y/2
    width = DIM[0]

    pfish = np.zeros((2, 1))
    pfish[0] = 0.5 * width + r * np.cos(theta)
    pfish[1] = 0.5 * width + r * np.sin(theta)
    return np.around(pfish).astype(int)

In [14]:
circular_i = np.zeros((1280, 1280), int)
circular_j = np.zeros((1280, 1280), int)
for i in range(1280):
    for j in range(2560):
        fishcoord = mapping(j, i)
        fishcoord[fishcoord >= 1280] = 1279
        fishcoord[fishcoord < 0] = 0
        circular_i[fishcoord[1], fishcoord[0]] = 1279 - i
        circular_j[fishcoord[1], fishcoord[0]] = j

In [2]:
circular_i = np.load('/home/shane/stitching/circular_i.npy')
circular_j = np.load('/home/shane/stitching/circular_j.npy')

In [15]:
img = cv2.imread('/home/shane/test_transformed.jpg')
# new_img = np.zeros((1280, 1280, 3), np.uint8)
new_img = img[circular_i, circular_j]
# new_img = new_img[row60, col60]
# cv2.imwrite('/home/shane/test_circular.jpg', new_img)
cv2.imshow('test', new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
new_img

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [5]:
def create_circular_mask(h, w, center=None, radius=None):

        if center is None: # use the middle of the image
            center = (int(w/2), int(h/2))
        if radius is None: # use the smallest distance between the center and image walls
            radius = min(center[0], center[1], w-center[0], h-center[1])

        Y, X = np.ogrid[:h, :w]
        dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
        col, row = np.meshgrid(np.arange(w), np.arange(h))
        theta = np.arctan2(h/2 - row, col-w/2)

        mask = np.logical_and(theta >= np.pi / 3, theta <= 2 * np.pi / 3)
        mask = np.logical_and(mask, dist_from_center <= radius)
        return mask

In [6]:
mask = create_circular_mask(1280, 1280, radius=630)

In [7]:
col, row = np.meshgrid(np.arange(1280), np.arange(1280))

col60 = rotate(col, -60, reshape=False)
row60 = rotate(row, -60, reshape=False)
